# 지도학습 기반의 대조학습

**지은이**: [Khalid Salama](https://www.linkedin.com/in/khalid-salama-24403144/)<br>
**옮긴이**: [Janghoo Lee](https://www.linkedin.com/in/janghoo-lee-25212a1a0/)<br>
**원본노트북:** [Supervised Contrastive Learning](https://keras.io/examples/vision/supervised-contrastive-learning/)<br>
**원본작성일:** 2020/11/30<br>
**최종수정일:** 2020/11/30<br>
**번역일:** 2021/09/04<br>
**번역최종수정일:** 2021/10/11<br>
**설명:** 지도학습 기반의 대조학습<sub>contrastive learning</sub>을 사용해서 이미지 분류 문제를 풀어 봅니다.

**선행 추천 노트북** <br>

1. 
Eng : [Semi-supervised image classification using contrastive pretraining with SimCLR](https://keras.io/examples/vision/semisupervised_simclr/) <br>
Kor : 준지도학습 기반의 대조적 사전학습 모델 (SimCLR) 을 이용한 이미지 분류
2.
Eng : [Image similarity estimation using a Siamese Network with a contrastive loss](https://keras.io/examples/vision/siamese_contrastive/)

<br>

*이 노트북은 2021 Open Source Contribution Contribution Academy, Keras Korea 의 지원을 받아 제작되었습니다. 한국어로 옮겨진 노트북은 이해를 돕기 위해 원본 노트북에서 제공하는 설명에 대해 추가적인 내용이 들어가 있음을 알립니다. 원문 설명은 [원본 노트북](https://keras.io/examples/vision/simsiam/)을 참고하세요.*

## 들어가며

기존에 널리 사용되던 크로스엔트로피 손실 기반의 지도학습 방식을 뛰어넘은 최근 학습 방식이 있습니다. 바로 대조학습<sub>contrastive learning</sub> 이라는 방법입니다. 대조학습은 주로 자기지도학습<sub>self-supervised</sub> 중심으로 연구가 진행되어 왔습니다. [MoCo](https://arxiv.org/abs/1911.05722) 나 [SimCLR](https://arxiv.org/abs/2002.05709) 같은 연구들이 이에 속합니다. 하지만 이 노트북에서 소개할 내용은 [지도학습 기반의 대조학습](https://arxiv.org/abs/2004.11362) (Prannay Khosla et al.) 입니다. 이미지 분류모델을 지도학습 기반의 대조학습으로 학습시키려면 아래와 같은 과정을 따라야 합니다.

1. 모델에 입력된 이미지를 잘 표현하는 벡터를 만들어내는 인코더를 학습시킵니다. 이때 인코더가 해야 하는 일은 대략적으로 다음과 같습니다. 범주 A 에 속하는 이미지를 `{a1, a2, a3, ...}` 라고 하고, 범주 B 에 속하는 이미지를 `{b1, b2, b3, ...}` 라고 해 봅시다.
    - `I`. 동일한 범주의 이미지에 대한 표현 벡터 쌍을 (a1, a2), (a1, a3), ... 이라고 하겠습니다.
    - `II`. 다른 범주의 이미지에 대한 표현 벡터 쌍을 (a1, b1), (a1, b2), ... 이라고 하겠습니다.
    - `III`. 이때, 모델은 `I`. 벡터 쌍 `(a1, a2)`, 또는 `(a1, a3)`, ... 등이 가까운 코사인거리를 가지고, `II`. 두 벡터 `(a1, b1)`, 또는 `(a1, b2)`, ... 등이 `I`. 에 비해 상대적으로 높은 코사인거리를 가지도록 학습합니다. 
2. 그 다음, 훈련이 불가능하도록 동결시킨<sub>frozen</sub> 인코더의 끝단에, 인코더가 생성하는 표현 벡터를 입력받아 이미지의 클래스를 구분해내는 분류기 레이어를 추가로 붙이고 학습시킵니다.

이 노트북의 예제를 실행해 보기 위해서는 [TensorFlow Addons]((https://www.tensorflow.org/addons)) 가 필요합니다. 이 커맨드를 통해 다운받도록 합니다.

```python
pip install tensorflow-addons
```

In [ ]:
# 구글 코랩 (Google COLAB) 환경이라면 아랫줄 코드를 주석 해제한 뒤 셀을 실행합니다.
!pip install tensorflow-addons

## 환경 준비하기

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

## 데이터 준비하기

In [ ]:
num_classes = 10
input_shape = (32, 32, 3)

# 훈련 데이터 세트와 평가 데이터 세트를 로드합니다.
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# 훈련 데이터 세트와 평가 데이터 세트의 모양을 확인합니다.
print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")

x_train shape: (50000, 32, 32, 3) - y_train shape: (50000, 1)
x_test shape: (10000, 32, 32, 3) - y_test shape: (10000, 1)


## 이미지 변형 정의하기

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.02),
        layers.RandomWidth(0.2),
        layers.RandomHeight(0.2),
    ]
)

# 일부 레이어는 (이 코드에서는 Normalization 레이어) 내부적으로 상태를 가지고 있습니다.
# 이러한 상태는 데이터세트에 맞게 미리 설정되어야 합니다.
# https://keras.io/guides/preprocessing_layers/ 을 참고하세요.
data_augmentation.layers[0].adapt(x_train)

## 인코더 모델 만들기

인코더는 입력 이미지를 받아서 2048 차원의 특징 벡터를 만들어냅니다.

In [ ]:
def create_encoder():
    resnet = keras.applications.ResNet50V2(
        include_top=False, weights=None, input_shape=input_shape, pooling="avg"
    )

    inputs = keras.Input(shape=input_shape)
    augmented = data_augmentation(inputs)
    outputs = resnet(augmented)
    model = keras.Model(inputs=inputs, outputs=outputs, name="cifar10-encoder")
    return model


encoder = create_encoder()
encoder.summary()

Model: "cifar10-encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
sequential_1 (Sequential)    (None, None, None, 3)     7         
_________________________________________________________________
resnet50v2 (Functional)      (None, 2048)              23564800  
Total params: 23,564,807
Trainable params: 23,519,360
Non-trainable params: 45,447
_________________________________________________________________


## 분류 모델 만들기

분류 모델은 아까 만들었던 인코더에 완전연결층과 소프트맥스층을 추가적으로 붙여서 완성합니다.

In [ ]:
def create_classifier(encoder, trainable=True):

    for layer in encoder.layers:
        layer.trainable = trainable

    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs)
    features = layers.Dropout(dropout_rate)(features)
    features = layers.Dense(hidden_units, activation="relu")(features)
    features = layers.Dropout(dropout_rate)(features)
    outputs = layers.Dense(num_classes, activation="softmax")(features)

    model = keras.Model(inputs=inputs, outputs=outputs, name="cifar10-classifier")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )
    return model

## 실험 1: 기준이 되는 분류 모델 만들어보기

기존의 분류 모델에서는 인코더에 분류기를 붙여 크로스엔트로피 손실로 인코더와 분류기 전체를 학습했고, 최근 제안된 지도학습 방식의 대조학습에 따르면 대조학습 방식으로 사전학습된 인코더에 분류기(완전연결층과 소프트맥스층) 를 붙여 완성한다고 했습니다.

이 실험에서는 인코더에 분류기를 붙여 크로스엔트로피 손실로 인코더와 분류기 전체를 학습하는 일반적으로 널리 사용되는 방식의 분류기를 만들 것입니다.

In [ ]:
learning_rate = 0.001
batch_size = 265
hidden_units = 512
projection_units = 128
num_epochs = 50
dropout_rate = 0.5
temperature = 0.05

encoder = create_encoder()
classifier = create_classifier(encoder)
classifier.summary()

history = classifier.fit(x=x_train, y=y_train, batch_size=batch_size, epochs=num_epochs)

accuracy = classifier.evaluate(x_test, y_test)[1]
print(f"Test accuracy: {round(accuracy * 100, 2)}%")

Model: "cifar10-classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
cifar10-encoder (Functional) (None, 2048)              23564807  
_________________________________________________________________
dropout_2 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               1049088   
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                5130      
Total params: 24,619,025
Trainable params: 24,573,578
Non-trainable params: 45,447
_______________________________

## 실험 2: 지도학습 방식의 대조학습

이제부터는 모델을 두 단계로 나누어 훈련시킵니다. 

1. 첫 단계에는, [Prannay Khosla et al.](https://arxiv.org/abs/2004.11362) 에서 제안한 대로, 인코더가 크로스엔트로피 손실이 아닌, 대조학습 손실을 최적화시키도록 사전학습시킵니다.
2. 두 번째 단계에는, 사전학습된 인코더를 사용하는 분류기의 가중치만 크로스엔트로피 손실을 최적화하도록 학습시킵니다.

*역주 : 이 노트북에서는 논문이 제안하는 loss 를 정확히 재구현하지는 않고, 지도학습 기반의 대조학습이라는 컨셉트만 유지합니다. 이를 간단히 구현하기 위해 n pair loss 를 사용하는데, 이에 대해서 더 궁금하다면 [이 블로그(영문)](https://towardsdatascience.com/contrasting-contrastive-loss-functions-3c13ca5f055e) 을 참고하세요.* 

### 1. 지도학습 방식의 대조 손실 함수

In [ ]:
class SupervisedContrastiveLoss(keras.losses.Loss):
    def __init__(self, temperature=1, name=None):
        super(SupervisedContrastiveLoss, self).__init__(name=name)
        self.temperature = temperature

    def __call__(self, labels, feature_vectors, sample_weight=None):
        
        # 128 차원으로 투영된 b 개의 특징벡터들 [b, 128]
        feature_vectors_normalized = tf.math.l2_normalize(feature_vectors, axis=1)
        
        # 코사인거리 [b, b]
        cosine_sim = tf.matmul(feature_vectors_normalized, tf.transpose(feature_vectors_normalized)) # [b, 128] * [128, b] = [b, b]
        # 참고 : 코사인거리 i 행 j 열이 나타내는 값은 i 번째 이미지와 j 번째 이미지의 코사인거리를 나타냅니다.
        logits = tf.divide(cosine_sim, self.temperature,)

        # npairs_loss(y_true shape : [b,], y_pred shape:[b, b])
        # n pair loss 를 적용합니다.
        return tfa.losses.npairs_loss(tf.squeeze(labels), logits)


def add_projection_head(encoder):
    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs) # features : 인코더가 만든 특징 벡터 [b, 2048]
    outputs = layers.Dense(projection_units, activation="relu")(features) # outputs : 투영된 특징 벡터 [b, 128]
    model = keras.Model(
        inputs=inputs, outputs=outputs, name="cifar-encoder_with_projection-head"
    )
    return model

### 2. 인코더 사전학습

In [ ]:
encoder = create_encoder()

encoder_with_projection_head = add_projection_head(encoder)
encoder_with_projection_head.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=SupervisedContrastiveLoss(temperature),
)

encoder_with_projection_head.summary()

history = encoder_with_projection_head.fit(
    x=x_train, y=y_train, batch_size=batch_size, epochs=num_epochs,
)

Model: "cifar-encoder_with_projection-head"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
cifar10-encoder (Functional) (None, 2048)              23564807  
_________________________________________________________________
dense_5 (Dense)              (None, 128)               262272    
Total params: 23,827,079
Trainable params: 23,781,632
Non-trainable params: 45,447
_________________________________________________________________
Epoch 1/50
189/189 [==============================] - 20s 81ms/step - loss: 5.3682
Epoch 2/50
189/189 [==============================] - 16s 83ms/step - loss: 5.1349
Epoch 3/50
189/189 [==============================] - 15s 82ms/step - loss: 5.0188
Epoch 4/50
189/189 [==============================] - 15s 80ms/step - loss: 4.9108
Epoch 5/50
18

### 3. 훈련되지 않도록 가중치가 고정된 인코더의 결과를 사용하는 분류기 훈련

In [ ]:
classifier = create_classifier(encoder, trainable=False)

history = classifier.fit(x=x_train, y=y_train, batch_size=batch_size, epochs=num_epochs)

accuracy = classifier.evaluate(x_test, y_test)[1]
print(f"Test accuracy: {round(accuracy * 100, 2)}%")

Epoch 1/50
189/189 [==============================] - 7s 26ms/step - loss: 0.3634 - sparse_categorical_accuracy: 0.8994
Epoch 2/50
189/189 [==============================] - 5s 26ms/step - loss: 0.3038 - sparse_categorical_accuracy: 0.9100
Epoch 3/50
189/189 [==============================] - 5s 26ms/step - loss: 0.2902 - sparse_categorical_accuracy: 0.9115
Epoch 4/50
189/189 [==============================] - 5s 26ms/step - loss: 0.2894 - sparse_categorical_accuracy: 0.9093
Epoch 5/50
189/189 [==============================] - 5s 26ms/step - loss: 0.2775 - sparse_categorical_accuracy: 0.9151
Epoch 6/50
189/189 [==============================] - 5s 26ms/step - loss: 0.2791 - sparse_categorical_accuracy: 0.9124
Epoch 7/50
189/189 [==============================] - 5s 26ms/step - loss: 0.2789 - sparse_categorical_accuracy: 0.9134
Epoch 8/50
189/189 [==============================] - 5s 26ms/step - loss: 0.2765 - sparse_categorical_accuracy: 0.9134
Epoch 9/50
189/189 [====================

검증 데이터셋 기준으로 실험1 에서 만든 모델을 통해 얻었던 결과보다 더 좋은 성능을 낼 수 있다는 것을 보여줍니다.

## 마치며

실험에서 보았듯, 지도학습 방식의 대조학습을 사용한다면 비슷한 실험조건(훈련 에폭수 등...) 에서 전통적으로 사용되던 성능보다 더 우수한 성능을 얻을 수 있게 됩니다. 대조학습은 이 노트북에서 사용한 모델보다 훈련이 더 어렵고 복잡한 아키텍처에서도 잘 동작하고, 다중 클래스 분류같이 단순 이미지 분류보다 확장되고 복잡해진 작업에서도 잘 동작합니다.

이 훈련 방법을 더 효율적으로 사용하기 위해서는 배치사이즈를 늘리고, 분류기를 더 깊게 쌓는 것이 도움이 될 수 있습니다. 더 자세한 내용이 궁금하다면, 논문 [Supervised Contrastive Learning](https://arxiv.org/abs/2004.11362) 을 참고하세요.

